In [1]:
import camelot
import pandas as pd
import openpyxl
import os
# tables = camelot.read_pdf('epd1388_2021.pdf', pages='10',process_background=True)
# tables = camelot.read_pdf('./IBU-09-20/New/14632.pdf', pages='6', split_text=True,table_areas=["69.60548391889084,500.088593765987,525.3099336281931,403.96614296894364"])
# print(len(tables))

# tables = camelot.read_pdf('./New/14699.pdf', pages='all')
# print(len(tables))

In [2]:
headers = [
    "impactGWP100_kgCO2e",
    "impactODP_kgCFC11e",
    "impactAP_kgSO2e",
    "impactEP_kgPO4e",
    "impactPOCP_kgEthenee",
    "impactADPElements_kgSbe",
    "impactADPFossilFuels_MJ",
    
    "renewablesUsedAsEnergy_MJ",
    "renewablesUsedAsMaterial_MJ",
    "embodiedRenewableEnergy_MJ",
    "nonRenewablesUsedAsEnergy_MJ",
    "nonRenewablesUsedAsMaterial_MJ",
    "embodiedEnergy_MJ",
    
    "recyclingMaterialUse_kg",
    "renewableRecylingFuelUse_MJ",
    "nonRenewableRecylingFuelUse_MJ",
    "cleanWaterNetUse_m3",
    "wasteHazardous_kg",
    "wasteNonHazardous_kg",
    "wasteRadioactive_kg",
    "reusableMaterialsOutput_kg",
    "recyclableMaterialsOutput_kg",
    "energyMaterialsOutput_kg",
    "exportedEnergyOutput_MJ_thermal",
    "exportedEnergyOutput_MJ_electric",
    "exportedEnergyOutput_MJ",
    
    "traciGWP_kgCO2e",
    "traciODP_kgCFC11e",
    "traciAP_kgSO2e",
    "traciEP_kgNe",
    "traciPOCP_kgO3e",
    "traciNRPE_MJ"
]
columns = {                                                                                                   
  "impactGWP100_kgCO2e": "(^GWP - Total$)|(^GWP$)|(^Global warming potential$)|(^Globales Erwärmungspotenzial ­ total$)|(^Globales Erwärmungspotenzial$)",
  "impactODP_kgCFC11e": "(^ODP$)|(^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)",
  "impactAP_kgSO2e": "(^AP$)|(^AP Air$)|(^Versauerungspotenzial von Boden und Wasser$)|(^Acidification potential of land and water$)",
  "impactEP_kgPO4e": "(^\*EP - Freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)",
  "impactPOCP_kgEthenee": "(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical \noxidants)",
  "impactADPElements_kgSbe": "(^ADPE$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nnicht fossile Ressourcen$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nnicht fossile Ressourcen$)|(^Abiotic depletion potential for non-fossil resources)",
  "impactADPFossilFuels_MJ": "(^ADPF$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nfossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nfossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)",
    
  "renewablesUsedAsEnergy_MJ": "(^PERE$)|(^Erneuerbare Primärenergie als Energieträger$)|(^Renewable primary energy as energy carrier$)",
  "renewablesUsedAsMaterial_MJ": "(^PERM$)|(^Erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Renewable primary energy resources as material utilization$)",
  "embodiedRenewableEnergy_MJ": "(^PERT$)|(^Total erneuerbare Primärenergie$)|(^Total use of renewable primary energy resources$)",
  "nonRenewablesUsedAsEnergy_MJ": "(^PENRE$)|(^Nicht-erneuerbare Primärenergie als Energieträger$)|(^Nicht\xaderneuerbare Primärenergie als Energieträger$)|(^Non-renewable primary energy as energy carrier$)",
  "nonRenewablesUsedAsMaterial_MJ": "(^PENRM$)|(^Nicht-erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Nicht\xaderneuerbare Primärenergie zur stofflichen Nutzung$)|(^Non-renewable primary energy as material utilization$)",
  "embodiedEnergy_MJ": "(^PENRT$)|(^Total nicht erneuerbare Primärenergie$)|(^Total use of non-renewable primary energy resources$)",
  "recyclingMaterialUse_kg": "(^SM$)|(^Einsatz von Sekundärstoffen$)|(^Use of secondary material$)",
  "renewableRecylingFuelUse_MJ": "(^RSF$)|(^Erneuerbare Sekundärbrennstoffe$)|(^Use of renewable secondary fuels$)",
  "nonRenewableRecylingFuelUse_MJ": "(^NRSF$)|(^Nicht-erneuerbare Sekundärbrennstoffe$)|(^Nicht\xaderneuerbare Sekundärbrennstoffe$)|(^Use of non-renewable secondary fuels$)",
  "cleanWaterNetUse_m3": "(^FW$)|(^Nettoeinsatz von Süßwasserressourcen$)|(^Use of net fresh water$)",
    
  "wasteHazardous_kg": "(^HWD$)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)",
  "wasteNonHazardous_kg": "(^NHWD$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)",
  "wasteRadioactive_kg": "(^RWD$)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)",
  "reusableMaterialsOutput_kg": "(^CRU$)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)",
  "recyclableMaterialsOutput_kg": "(^MFR$)|(^Stoffe zum Recycling$)|(^Materials for recycling$)",
  "energyMaterialsOutput_kg": "(^MER$)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)",
  "exportedEnergyOutput_MJ_electric": "(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)",
  "exportedEnergyOutput_MJ_thermal": "(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)",
  "exportedEnergyOutput_MJ": "(^EE$)",
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}

In [3]:
def make_excel_file(files_data):
    book = openpyxl.Workbook()
    filename = 'test_ibu.xlsx'
    book.save(filename)

    sheet = book['Sheet']
    sheet.cell(row=1, column=1).value = 'Filename'
    sheet.cell(row=1, column=2).value = 'EPD_Number'
    sheet.cell(row=1, column=3).value = 'DownloadLink'
    sheet.cell(row=1, column=4).value = 'Stages'
    
    count = 5
    for header in headers:
        sheet.cell(row=1, column=count).value = header
        count += 1
    
    current_row = 2
    for file_data in files_data:
#         print(file_data["file_name"])
#         print(file_data["stages"])
#         print("*******************")
        df = file_data["dataFrame"]
        base_added = False
        
#         if file_data["type"] == 1 :
        for stage in file_data["stages"]:
            if not base_added:
                sheet.cell(row=current_row, column=1).value = file_data["file_name"]
                sheet.cell(row=current_row, column=2).value = 'NO NUMBER CHECK IT!'
                sheet.cell(row=current_row, column=3).value = '/mnt/data/Downloads/2021-10-18/IBU/New/' + file_data["file_name"]
                base_added = True

            sheet.cell(row=current_row, column=4).value = stage
            header_counter = 0
            for header in headers:
                
#                 i = file_data["stages"].index(stage) + 2
#                 sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
#                 value = float(df[df[0].str.contains(columns2[header])][i].to_list()[0])
#                 #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
# #                 sheet.cell(row=current_row, column= 5 + header_counter).value = value
                try:
                    num_of_columns = len(df.columns)                    
                    i = file_data["stages"].index(stage) + (num_of_columns - len(file_data["stages"]))
                    sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
                    replaced = df[df[0].str.contains(columns[header])][i].to_list()[0].replace(',', '.')
                    if not replaced == 'IND':
                        try:
                            value = float(replaced)
                        except:
                            value = float(replaced.replace('\xad', '-'))
                    else:
                        value = replaced
                        
                    #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
                    sheet.cell(row=current_row, column= 5 + header_counter).value = value
                except:
                    print("**************************************ERRROR*****************************")
                    print(file_data["file_name"])
                    print(stage)
                    print(columns[header])
                header_counter= header_counter +1
            current_row = current_row + 1
        current_row = current_row + 1
            
    book.save(filename)
    book.close()

In [4]:
#files = os.listdir('./4')
files = ['epd4044.pdf', 'epd4051.pdf', 'epd4054.pdf','epd4061.pdf']

r = []
for file in files:
    tables = camelot.read_pdf('./4/' + file, pages='all')
    print("************ FILE NAME: " + file + " ****************")
    dataFrames = []
    file_info = {}
    stages = []
    for table in tables:
        df = table.df

        #dataFrames = []
        gwp = ['GWP-total', 'GWP - Total', 'Global warming potential', 'GWP', 'Globales Erwärmungspotenzial ­ total', 'Globales Erwärmungspotenzial']
        for g in gwp:
            if (g in df.values) and ('TRACI 2.1 Impact Assessment' not in df.values):
                dataFrames.append(df)


        pere = ['PERE', 'Renewable primary energy as energy carrier', 'Erneuerbare Primärenergie als Energieträger']
        for p in pere:
            if p in df.values:
                dataFrames.append(df)

        hwd = ['HWD', 'Hazardous waste disposed', 'Gefährlicher Abfall zur Deponie']
        for h in hwd:
            if h in df.values:
                dataFrames.append(df)


    try:
        rez = []
        for list in df[df.eq('Unit').any(1)].values:
            rez = rez + list.tolist()
        longest_list = []
        help = [longest_list.append(x) for x in rez if x not in longest_list]

        droped_nan = [e for e in longest_list if str(e) != 'nan']


        unit_index = droped_nan.index('Unit') + 1
        stages = droped_nan[unit_index:]
        print(stages)
    except:
        a = list(df[df.eq('Einheit').any(1)].values[0])
        unit_index = a.index('Einheit') + 1
        stages = a[unit_index:]




                
    result = pd.concat(dataFrames)
    file_info["dataFrame"] = result        
    file_info["file_name"] = file
    file_info["stages"] = stages
    
    r.append(file_info)
    
    #write_to_excel(dataFrame, stages)

make_excel_file(r)

# print(types)
# r

************ FILE NAME: epd4044.pdf ****************
['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']
************ FILE NAME: epd4051.pdf ****************
['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']
************ FILE NAME: epd4054.pdf ****************
['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']
************ FILE NAME: epd4061.pdf ****************
['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']
**************************************ERRROR*****************************
epd4044.pdf
A1-A3
(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)
**************************************ERRROR*****************************
epd4044.pdf
A1-A3
(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd4044.pdf
A1-A3
(^EE$)
**************************************ERRROR*****************************
epd4044.pdf
A1-A3
8
**************************************ERRROR*****************

C:\Users\mirol\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


**************************************ERRROR*****************************
epd4051.pdf
C3
(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)
**************************************ERRROR*****************************
epd4051.pdf
C3
(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd4051.pdf
C3
(^EE$)
**************************************ERRROR*****************************
epd4051.pdf
C3
8
**************************************ERRROR*****************************
epd4051.pdf
C3
12
**************************************ERRROR*****************************
epd4051.pdf
C3
6
**************************************ERRROR*****************************
epd4051.pdf
C3
7
**************************************ERRROR*****************************
epd4051.pdf
C3
10
**************************************ERRROR*****************************
epd4051.pdf
C3
17
*******************************

**************************************ERRROR*****************************
epd4061.pdf
C3
(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)
**************************************ERRROR*****************************
epd4061.pdf
C3
(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd4061.pdf
C3
(^EE$)
**************************************ERRROR*****************************
epd4061.pdf
C3
8
**************************************ERRROR*****************************
epd4061.pdf
C3
12
**************************************ERRROR*****************************
epd4061.pdf
C3
6
**************************************ERRROR*****************************
epd4061.pdf
C3
7
**************************************ERRROR*****************************
epd4061.pdf
C3
10
**************************************ERRROR*****************************
epd4061.pdf
C3
17
*******************************

In [8]:
['A1-A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D']

In [6]:
r
r[3]['file_name']

'epd4061.pdf'

In [7]:

test_df = r[3]['dataFrame']
rez = []
for list in test_df[test_df.eq('Unit').any(1)].values:
    rez = rez + list.tolist()
longest_list = []
help = [longest_list.append(x) for x in rez if x not in longest_list]

droped_nan = [e for e in longest_list if str(e) != 'nan']


unit_index = droped_nan.index('Unit') + 1
stages = droped_nan[unit_index:]
print(stages)

['A1-A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D', 'A5']


In [10]:
a = list(test_df[test_df.eq('Unit').any(1)].values[0])
unit_index = a.index('Unit') + 1
stages = a[unit_index:]

In [11]:
a

['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']

In [12]:
unit_index

2

In [13]:
stages

['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']

In [11]:
r[3]['file_name']

'epd4061.pdf'

In [6]:
test_df = r[3]['dataFrame']

In [14]:
droped_nan = []
lists = test_df[test_df.eq('Unit').any(1)].values
for row in lists:
    row_elements = [e for e in row if str(e) != 'nan']
    droped_nan.append(row_elements)
lengh_of_droped_nan = [len(list) for list in droped_nan]
longest_list = droped_nan[lengh_of_droped_nan.index(max(lengh_of_droped_nan))]

unit_index = longest_list.index('Unit') + 1
stages = longest_list[unit_index:]
print(stages)

['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']


In [15]:
longest_list

['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']

In [16]:
lengh_of_droped_nan

[9, 9, 10]

In [17]:
lists

array([['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'C1', 'C2', 'C3',
        'C4', 'D', nan],
       ['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'C1', 'C2', 'C3',
        'C4', 'D', nan],
       ['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'A5', 'C1', 'C2',
        'C3', 'C4', 'D']], dtype=object)

In [7]:
test_df

,0,1,2,3,4,5,6,7,8,9
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D,NaN
1,GWP - Fossil,kg CO2 eq,2.10,0.255,0.134,0.471,0.073,0.065,-0.006,NaN
2,GWP - Biogenic,kg CO2 eq,-0.511,185E-6,0.002,0.004,52.8E-6,129E-6,-13.7E-6,NaN
3,GWP - Luluc,kg CO2 eq,0.006,74.3E-6,66.3E-6,0.004,21.2E-6,18.1E-6,-1.57E-6,NaN
4,GWP - Total,kg CO2 eq,1.60,0.255,0.136,0.480,0.073,0.065,-0.006,NaN
5,ODP,kg CFC-11 eq,281E-9,59.9E-9,7.04E-9,13.3E-9,17.1E-9,26.7E-9,-95.9E-12,NaN
6,AP,mol H+ eq,0.009,0.001,0.001,0.003,306E-6,0.001,-19.7E-6,NaN
7,EP - Freshwater,kg P eq,0.001,18.0E-6,36.2E-6,497E-6,5.15E-6,6.66E-6,-814E-9,NaN
8,*EP - Freshwater,kg PO4 eq,0.002,55.2E-6,111E-6,0.002,15.8E-6,20.4E-6,-2.49E-6,NaN
9,EP - Marine,kg N eq,0.002,325E-6,135E-6,0.001,92.9E-6,213E-6,-3.46E-6,NaN


In [19]:
test_df[test_df.eq('Unit').any(1)]

,0,1,2,3,4,5,6,7,8,9
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D,NaN
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D,NaN
0,Impact \nCategory,Unit,A1-A3,A4,A5,C1,C2,C3,C4,D


In [22]:
test_df[test_df.eq('Unit').any(1)].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [25]:
test_df[test_df.eq('Unit').any(1)].values

array([['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'C1', 'C2', 'C3',
        'C4', 'D', nan],
       ['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'C1', 'C2', 'C3',
        'C4', 'D', nan],
       ['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'A5', 'C1', 'C2',
        'C3', 'C4', 'D']], dtype=object)

In [39]:

rez = []
for list in test_df[test_df.eq('Unit').any(1)].values:
    rez = rez + list.tolist()
rez
unique = []
help = [unique.append(x) for x in rez if x not in unique]
unique

['Impact \nCategory',
 'Unit',
 'A1-A3',
 'A4',
 'C1',
 'C2',
 'C3',
 'C4',
 'D',
 nan,
 'A5']

In [ ]:
columns = {                                                                                                   
    'impactGWP100_kgCO2e_fossil':'(^GWP - Fossil$)',
    'impactGWP100_kgCO2e_biogenic':'(^GWP - Biogenic$)',
    'impactGWP100_kgCO2e_luluc':'(^GWP - Luluc$)',
    'impactODP_WMO2014_kgCFC11e':'(^ODP$)',
    'impactAP_molHeq':'(^AP$)',
    'impactEPFreshwater_kgPO4e':'(^\*EP - Freshwater$)',
    'impactEPFreshwater_kgP':'(^EP - Freshwater$)',
    'impactEPMarine_kgNe':'(^EP - Marine$)',
    'impactEPTerrestrial_molNe':'(^EP - Terrestrial$)',
    'impactPOCP_kgNMVOCe':'(^POCP$)',
    'cleanWaterDeprivation_m3':'(^WDP$)',
    'particulateMatter_incidence':'(^PM$)',
    'ionisingRadiation_kgU235eq':'(^IR$)',
    'ecoToxicityFreshwater_CTUe':'(^ETP - FW$)',
    'humanToxicityCancer_CTUh':'(^HTTP - C$)',
    'humanToxicityNonCancer_CTUh':'(^HTTP - NC$)',
    'potentialSoilQualityIndex':'(^SQP$)',
    
  "renewablesUsedAsEnergy_MJ": "(^PERE$) ",
  "renewablesUsedAsMaterial_MJ": "(^PERM$) ",
  "embodiedRenewableEnergy_MJ": "(^PERT$) ",
  "nonRenewablesUsedAsEnergy_MJ": "(^PENRE$) ",
  "nonRenewablesUsedAsMaterial_MJ": "(^PENRM$) ",
  "embodiedEnergy_MJ": "(^PENRT$) ",
  "recyclingMaterialUse_kg": "(^SM$) ",
  "renewableRecylingFuelUse_MJ": "(^RSF$) ",
  "nonRenewableRecylingFuelUse_MJ": "(^NRSF$) ",
  "cleanWaterNetUse_m3": "(^FW$) ",
    
  "wasteHazardous_kg": "(^HWD$) ",
  "wasteNonHazardous_kg": "(^NHWD$) ",
  "wasteRadioactive_kg": "(^RWD$) ",
  "reusableMaterialsOutput_kg": "(^CRU$) ",
  "recyclableMaterialsOutput_kg": "(^MFR$) ",
  "energyMaterialsOutput_kg": "(^MER$) ",
  "exportedEnergyOutput_MJ_electric": "(^EEE$) ",
  "exportedEnergyOutput_MJ_thermal": "(^EET$) ",
  "exportedEnergyOutput_MJ": "",
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}

In [47]:
tables = camelot.read_pdf('./4/epd4054.pdf', pages='1', flavor='stream')
print(len(tables))

1


In [50]:
tables[0].df[1][3]

'S-P-04054'

In [7]:
rez = []
for list in test_df[test_df.eq('Unit').any(1)].values:
    rez = rez + list.tolist()
longest_list = []
help = [longest_list.append(x) for x in rez if x not in longest_list]

droped_nan = [e for e in longest_list if str(e) != 'nan']


unit_index = droped_nan.index('Unit') + 1
stages = droped_nan[unit_index:]
print(stages)
print(test_df[test_df.eq('Unit').any(1)].values)

['A1-A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D', 'A5']
[['Impact \nCategory' 'Unit' 'A1-A3' 'A4' 'C1' 'C2' 'C3' 'C4' 'D' nan]
 ['Impact \nCategory' 'Unit' 'A1-A3' 'A4' 'C1' 'C2' 'C3' 'C4' 'D' nan]
 ['Impact \nCategory' 'Unit' 'A1-A3' 'A4' 'A5' 'C1' 'C2' 'C3' 'C4' 'D']]


In [8]:
test_df[test_df[0].str.contains(columns['impactEP_kgPO4e'])]

C:\Users\mirol\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,0,1,2,3,4,5,6,7,8,9
8,*EP - Freshwater,kg PO4 eq,0.002,55.2E-6,111E-6,0.002,15.8E-6,20.4E-6,-2.49E-6,NaN


In [9]:
df2 = test_df.set_axis(['Impact \nCategory', 'Unit','A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D'], axis=1, inplace=False)
df2

,Impact \nCategory,Unit,A1-A3,A4,A5,C1,C2,C3,C4,D
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D,NaN
1,GWP - Fossil,kg CO2 eq,2.10,0.255,0.134,0.471,0.073,0.065,-0.006,NaN
2,GWP - Biogenic,kg CO2 eq,-0.511,185E-6,0.002,0.004,52.8E-6,129E-6,-13.7E-6,NaN
3,GWP - Luluc,kg CO2 eq,0.006,74.3E-6,66.3E-6,0.004,21.2E-6,18.1E-6,-1.57E-6,NaN
4,GWP - Total,kg CO2 eq,1.60,0.255,0.136,0.480,0.073,0.065,-0.006,NaN
5,ODP,kg CFC-11 eq,281E-9,59.9E-9,7.04E-9,13.3E-9,17.1E-9,26.7E-9,-95.9E-12,NaN
6,AP,mol H+ eq,0.009,0.001,0.001,0.003,306E-6,0.001,-19.7E-6,NaN
7,EP - Freshwater,kg P eq,0.001,18.0E-6,36.2E-6,497E-6,5.15E-6,6.66E-6,-814E-9,NaN
8,*EP - Freshwater,kg PO4 eq,0.002,55.2E-6,111E-6,0.002,15.8E-6,20.4E-6,-2.49E-6,NaN
9,EP - Marine,kg N eq,0.002,325E-6,135E-6,0.001,92.9E-6,213E-6,-3.46E-6,NaN


In [ ]:
df2 = test_df.rename({'a': 'X', 'b': 'Y'}, axis=1

In [24]:
t = test_df[0:5]
t = t.rename(['Impact \nCategory', 'Unit','A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D'], axis=1, inplace=False)

TypeError: 'list' object is not callable

In [14]:
t

,0,1,2,3,4,5,6,7,8,9
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D,NaN
1,GWP - Fossil,kg CO2 eq,2.10,0.255,0.134,0.471,0.073,0.065,-0.006,NaN
2,GWP - Biogenic,kg CO2 eq,-0.511,185E-6,0.002,0.004,52.8E-6,129E-6,-13.7E-6,NaN
3,GWP - Luluc,kg CO2 eq,0.006,74.3E-6,66.3E-6,0.004,21.2E-6,18.1E-6,-1.57E-6,NaN
4,GWP - Total,kg CO2 eq,1.60,0.255,0.136,0.480,0.073,0.065,-0.006,NaN


In [21]:
t[t.eq('Unit').any(1)].values.tolist()[0]

['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D', nan]

In [22]:
l =t[t.eq('Unit').any(1)].values.tolist()[0]

In [25]:
t

,0,1,2,3,4,5,6,7,8,9
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D,NaN
1,GWP - Fossil,kg CO2 eq,2.10,0.255,0.134,0.471,0.073,0.065,-0.006,NaN
2,GWP - Biogenic,kg CO2 eq,-0.511,185E-6,0.002,0.004,52.8E-6,129E-6,-13.7E-6,NaN
3,GWP - Luluc,kg CO2 eq,0.006,74.3E-6,66.3E-6,0.004,21.2E-6,18.1E-6,-1.57E-6,NaN
4,GWP - Total,kg CO2 eq,1.60,0.255,0.136,0.480,0.073,0.065,-0.006,NaN


,0,1,2,3,4,5,6,7,8,9
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D,NaN


0    Impact \nCategory
1         GWP - Fossil
2       GWP - Biogenic
3          GWP - Luluc
4          GWP - Total
Name: 0, dtype: object

In [39]:
d = t.rename(columns=df.iloc[0])
d

,Impact \nCategory,Unit,A1-A3,A4,A5,C1,C2,C3,C4,D
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D,NaN
1,GWP - Fossil,kg CO2 eq,2.10,0.255,0.134,0.471,0.073,0.065,-0.006,NaN
2,GWP - Biogenic,kg CO2 eq,-0.511,185E-6,0.002,0.004,52.8E-6,129E-6,-13.7E-6,NaN
3,GWP - Luluc,kg CO2 eq,0.006,74.3E-6,66.3E-6,0.004,21.2E-6,18.1E-6,-1.57E-6,NaN
4,GWP - Total,kg CO2 eq,1.60,0.255,0.136,0.480,0.073,0.065,-0.006,NaN


In [42]:
d[0:1]['C1']

0    C2
Name: C1, dtype: object

In [45]:
d[d['Impact \nCategory'].str.contains('GWP - Total')]['A1-A3']

4    1.60
Name: A1-A3, dtype: object

In [ ]:
d[d[0].str.contains('GWP - Total')][stage].to_list()[0].replace(',', '.')

In [ ]:
'impactGWP100_kgCO2e_fossil':'(^GWP - Fossil$)'
'impactGWP100_kgCO2e_biogenic':'(^GWP - Biogenic$)'
'impactGWP100_kgCO2e_luluc':'(^GWP - Luluc$)'
'impactODP_WMO2014_kgCFC11e':'(^ODP$)'
'impactAP_molHeq':'(^AP$)'
'impactEPFreshwater_kgPO4e':'(^\*EP - Freshwater$)'
'impactEPFreshwater_kgP':'(^EP - Freshwater$)'
'impactEPMarine_kgNe':'(^EP - Marine$)'
'impactEPTerrestrial_molNe':'(^EP - Terrestria$)'l
'impactPOCP_kgNMVOCe':'(^POCP$)'
'cleanWaterDeprivation_m3':'(^WDP$)'
'particulateMatter_incidence':'(^PM$)'
'ionisingRadiation_kgU235eq':'(^IR$)'
'ecoToxicityFreshwater_CTUe':'(^ETP - FW$)'
'humanToxicityCancer_CTUh':'(^HTTP - C$)'
'humanToxicityNonCancer_CTUh':'(^HTTP - NC$)'
'potentialSoilQualityIndex':'(^SQP$)'


In [51]:
df2

,Impact \nCategory,Unit,A1-A3,A4,A5,C1,C2,C3,C4,D
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D,NaN
1,GWP - Fossil,kg CO2 eq,2.10,0.255,0.134,0.471,0.073,0.065,-0.006,NaN
2,GWP - Biogenic,kg CO2 eq,-0.511,185E-6,0.002,0.004,52.8E-6,129E-6,-13.7E-6,NaN
3,GWP - Luluc,kg CO2 eq,0.006,74.3E-6,66.3E-6,0.004,21.2E-6,18.1E-6,-1.57E-6,NaN
4,GWP - Total,kg CO2 eq,1.60,0.255,0.136,0.480,0.073,0.065,-0.006,NaN
5,ODP,kg CFC-11 eq,281E-9,59.9E-9,7.04E-9,13.3E-9,17.1E-9,26.7E-9,-95.9E-12,NaN
6,AP,mol H+ eq,0.009,0.001,0.001,0.003,306E-6,0.001,-19.7E-6,NaN
7,EP - Freshwater,kg P eq,0.001,18.0E-6,36.2E-6,497E-6,5.15E-6,6.66E-6,-814E-9,NaN
8,*EP - Freshwater,kg PO4 eq,0.002,55.2E-6,111E-6,0.002,15.8E-6,20.4E-6,-2.49E-6,NaN
9,EP - Marine,kg N eq,0.002,325E-6,135E-6,0.001,92.9E-6,213E-6,-3.46E-6,NaN


In [53]:
df2.eq('GWP').any(1)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
dtype: bool

In [55]:
df[0].str.contains('(^GWP*)| (^Global*)')

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
Name: 0, dtype: bool